# Borrar Documentos - API

Este script es responsable de eliminar documentos de un servidor remoto utilizando la API de un servicio web. 

Además, si se especifica, realiza una copia de seguridad del documento antes de eliminarlo.

El script lee un archivo CSV con los IDs y nombres de los documentos a eliminar, y ejecuta la eliminación en base a esos datos.


### Lo primero que debemos hacer es obtener el token de la empresa


In [1]:
# Imports
import requests
import csv
import os
import json
from dotenv import load_dotenv

In [3]:
# Cargar el token desde la configuracion
load_dotenv(override=True)

token = os.getenv('TOKEN_WOFFU')

In [4]:
if not token:
  raise ValueError("El token no puede ser nulo o vacío")
else:
    print (f"Token configurado correctamente.")


Token configurado correctamente.


## Verificar que podemos conectarnos correctamente a la API

- Request URL:
    https://spacexfree.woffu.dev/api/companies/{CompanyId}/settings
- Request Method:
    GET


In [5]:
# URL base
base_url = "https://spacexfree.woffu.dev"

# Endpoint específico de la API
endpoint = "/api/companies/79843/settings"

# Construir la URL completa combinando base_url y endpoint
url = base_url + endpoint

# Cabeceras de la solicitud, incluyendo el token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"  # Indica que esperamos recibir una respuesta en formato JSON
}

# Realizar la solicitud GET
response = requests.get(url, headers=headers)

# Comprobar el estado de la respuesta
if response.status_code == 200:
    # Si la respuesta es exitosa, convertir la respuesta JSON a un diccionario y mostrarla con formato
    response_json = response.json()
    print(f"Respuesta exitosa:")                
    print(json.dumps(response_json, indent=4)) 
else:
    # Si hubo un error en la solicitud, mostrar el código de estado
    print(f"Error {response.status_code}: {response.text}")  

Respuesta exitosa:
{
    "$id": "1",
    "CompanyId": 79843,
    "DefaultRoleId": 3,
    "HoursFormatted": true,
    "UseDiary": true,
    "UseFtp": false,
    "WoffuBay": false,
    "Benefits": false,
    "UseSignByMotive": false,
    "EndContractAction": null,
    "UseComplaintChannel": false,
    "ComplaintChannelTermsAndConditionsUrl": null,
    "ConsumptionWoffuBay": false
}


## Definir la fuctions para remover documentos de una empresa

Datos necesarios:
    - DocumentId
    - DocumentName

In [6]:
def remove_document(document_id, document_name, 
                    token, 
                    host_http="midatel.woffu.com", 
                    url="https://midatel.woffu.com/api/documents/bulk/delete", 
                    backup=True, 
                    backup_url="https://midatel.woffu.com/api/documents"):
    headers = {
        "sec-ch-ua-platform": '"Windows"',
        "Authorization": f"Bearer {token}",
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json",
        "sec-ch-ua-mobile": "?0",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Dest": "empty",
        "host": host_http
    }

    body = {
        "ids": [document_id],
        "unselectedIds": [],
        "allDocuments": False,
        "visible": False,
        "tagId": None
    }

    if backup:
        print(f"Backing up document: {document_name}")
        backup_url = f"{backup_url}/{document_id}/download2"
        response = requests.get(backup_url, headers=headers)

        # Guardar el archivo respaldado
        backup_path = os.path.join("C:\\temp\\BackupFiles", document_name)
        with open(backup_path, 'wb') as file:
            file.write(response.content)

    # Realizar la eliminación del documento
    response = requests.post(url, headers=headers, json=body)
    
    if response.status_code == 200:
        print(f"Document: {document_name} was successfully deleted.")
    else:
        print(f"Error deleting document: {document_name}. Status Code: {response.status_code}")
    print("************************************************")




In [7]:
remove_document(188135,  
                "Facilita el registro de jornada de tu equipo con nuevos métodos de fichaje.png", 
                token, 
                "spacexfree.woffu.dev",
                "https://spacexfree.woffu.dev/api/documents/bulk/delete",
                True,
                "https://spacexfree.woffu.dev/api/documents")

Backing up document: Facilita el registro de jornada de tu equipo con nuevos métodos de fichaje.png
Error deleting document: Facilita el registro de jornada de tu equipo con nuevos métodos de fichaje.png. Status Code: 403
************************************************
